In [0]:
%pip install databricks-feature-engineering

In [0]:
%restart_python

In [0]:
%pip install databricks-feature-engineering
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from mlflow.tracking.client import MlflowClient
from pyspark.sql.functions import current_timestamp, lit

In [0]:
# Load data from Delta table
wine_df = spark.table("jpmc_group_catalog.mlops.wine_data_poc_dabs")

# Prepare inference data with entity key and raw acidity columns for UDF computation 
inference_df = wine_df.select("wine_id", "citric_acid", "fixed_acidity", "volatile_acidity")

# Prepare actual labels for comparison (optional for evaluation)
label_df = wine_df.select("wine_id", "quality")

# Add timestamp for monitoring and tracking predictions
inference_df = inference_df.withColumn("timestamp", current_timestamp())


In [0]:


model_name = "jpmc_group_catalog.mlops.wine_rf_model_dabs"
mlflow_client = MlflowClient()
def get_latest_model_version (model_name):
    latest_version = 1
    for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
        version_int = int(mv.version)
        if version_int > latest_version:
           latest_version = version_int
    return latest_version

latest_model_version = get_latest_model_version(model_name)
print(f"Latest model version: {latest_model_version}")


In [0]:


fe = FeatureEngineeringClient()

# Score batch using registered model
predictions_df = fe.score_batch(
     model_uri=f"models:/{model_name}/{latest_model_version}",
     df=inference_df
)

# Add model version for lineage
predictions_df = predictions_df.withColumn("model_id", lit(latest_model_version)) 
predictions_df = predictions_df.join(label_df, on="wine_id", how="left")




In [0]:

# Define output table path

predictions_table = "jpmc_group_catalog.mlops.wine_predictions_v2_dabs"

# Save predictions to Delta table (schema evolution enabled)
predictions_df.write.format("delta").mode("overwrite").option("mergeschema", "true").saveAsTable(predictions_table)

# Display for verification
display(predictions_df)

In [0]:
%sql

-- Run in a SQL cell
ALTER TABLE `jpmc_group_catalog`.`mlops`.`wine_predictions_v2_dabs`
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);
